In [ ]:
from pathlib import Path
import shutil

scratchpad_dir = Path("../scratchpad")

# Subdirectories to clean
subdirs = ["plots", "data", "notes"]

cleaned = []
for subdir in subdirs:
    folder = scratchpad_dir / subdir
    if folder.exists():
        # Remove all files in the folder
        for item in folder.iterdir():
            if item.is_file():
                item.unlink()
                cleaned.append(str(item))
            elif item.is_dir():
                shutil.rmtree(item)
                cleaned.append(str(item) + "/")
    else:
        # Create the folder if it doesn't exist
        folder.mkdir(parents=True, exist_ok=True)

if cleaned:
    print(f"🧹 Cleaned {len(cleaned)} items from scratchpad:")
    for item in cleaned:
        print(f"   - {item}")
else:
    print("✨ Scratchpad folders are already empty")

print(f"\n📁 Empty folders preserved:")
for subdir in subdirs:
    print(f"   - scratchpad/{subdir}/")

# Credibility Sub-Agent Test Notebook

This notebook tests the credibility sub-agent, which is responsible for:
- **Claim Verification**: Checking if claims are supported by reliable evidence
- **Source Assessment**: Evaluating the trustworthiness of sources used
- **Consistency Checking**: Looking for contradictions or inconsistencies
- **Bias Identification**: Noting potential biases in sources or analysis

The main agent uses this sub-agent to:
- Verify research findings from the web research agent
- Assess source quality and reliability
- Identify potential misinformation or biased reporting
- Rate overall trustworthiness of gathered information

## Setup

In [ ]:
import sys
import importlib

sys.path.insert(0, '..')  # Add parent directory to path

# Import and reload to pick up any changes to the agent
from agents import credibility_agent
importlib.reload(credibility_agent)
from agents.credibility_agent import credibility_agent_graph

from langgraph.types import Command
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✅ Credibility agent loaded successfully (with latest changes)")

## Helper Function to Test the Agent

In [ ]:
def test_credibility_agent(user_message: str, thread_id: str = "test-thread"):
    """
    Test the credibility agent with a user message.
    
    Args:
        user_message: The content/claims for the credibility agent to assess
        thread_id: Unique thread ID for this conversation
    
    Returns:
        The agent's final response message
    """
    print("="*80)
    print(f"CONTENT TO VERIFY:\n{user_message}")
    print("="*80)
    print("\nAssessing credibility...\n")
    
    # Invoke the agent with a fresh conversation (NOT resuming)
    result = credibility_agent_graph.invoke(
        {"messages": [("user", user_message)]},
        config={"configurable": {"thread_id": thread_id}}
    )
    
    # Get the final AI message
    final_message = result["messages"][-1].content
    
    print("="*80)
    print("CREDIBILITY AGENT ASSESSMENT:")
    print("="*80)
    print(final_message)
    print("\n" + "="*80)
    print("✅ Assessment completed.")
    print("="*80 + "\n")
    
    return final_message

---
# Example 1: Simple Claim Verification (Simple)

**Context**: A simple factual claim about a company that needs verification.

**Sub-agent role**: Verify a straightforward claim using web search to find corroborating evidence.

In [ ]:
# Example 1: Simple claim verification
example_1_message = """Please verify the following claim:

"Apple Inc. is the most valuable publicly traded company in the world by market capitalization."

Check if this claim is accurate as of the current date.
"""

# Use a fresh thread ID to avoid any cached responses
import time
response_1 = test_credibility_agent(example_1_message, thread_id=f"example-1-{int(time.time())}")

---
# Example 2: Research Output Assessment (Medium)

**Context**: The web research agent has produced findings about a stock that need verification.

**Sub-agent role**: Assess the credibility of multiple claims and sources from a research report.

In [ ]:
# Example 2: Research output assessment
example_2_message = """Please assess the credibility of this research output:

---
## NVIDIA Stock Analysis Summary

**Key Findings:**
1. NVIDIA's data center revenue grew over 200% year-over-year in their most recent quarter
2. The company has captured approximately 80-90% of the AI chip market
3. Major tech companies including Microsoft, Google, and Amazon are all using NVIDIA GPUs for AI workloads
4. The stock has risen over 200% in 2024

**Sources Used:**
- NVIDIA Investor Relations
- Reuters
- Bloomberg
- TechCrunch

**Original Question:** "Why has NVIDIA stock performed so well recently?"
---

Verify the key claims and assess whether this research adequately answers the original question.
"""

response_2 = test_credibility_agent(example_2_message, thread_id=f"example-2-{int(time.time())}")

---
# Example 3: Contradictory Information Assessment (Complex)

**Context**: Research has uncovered conflicting information about a stock from different sources.

**Sub-agent role**: Analyze contradictory claims, assess source reliability, and determine which narrative is more credible.

In [ ]:
# Example 3: Contradictory information assessment
example_3_message = """Please assess the credibility of these conflicting research findings:

---
## Tesla Stock Analysis - Conflicting Views

**Bullish Research (Source: Tesla fan blog "TeslaDaily.com"):**
- "Tesla will dominate the robotaxi market by 2025"
- "Full Self-Driving is already safer than human drivers"
- "Tesla's margins are industry-leading and sustainable"
- "Competition from legacy automakers is irrelevant"

**Bearish Research (Source: Short-seller report from "Hindenburg Research"):**
- "Tesla's FSD claims are exaggerated and potentially dangerous"
- "Margins are declining due to price cuts and competition"
- "Chinese EV makers are taking significant market share"
- "Robotaxi timeline has been repeatedly delayed"

**Neutral Research (Source: Goldman Sachs equity research):**
- "Tesla maintains technology leadership but faces margin pressure"
- "FSD progress is notable but regulatory approval timeline uncertain"
- "Competition is intensifying but Tesla's brand remains strong"

**Original Question:** "What is the outlook for Tesla stock over the next 12 months?"
---

Please:
1. Assess the credibility of each source
2. Identify which claims can be verified vs which are speculative
3. Note any potential biases in each source
4. Determine if this research provides a balanced view to answer the original question
"""

response_3 = test_credibility_agent(example_3_message, thread_id=f"example-3-{int(time.time())}")

---
# Notes

## Expected Outputs

For each example, the credibility agent should provide:

### Claim Verification
- VERIFIED / PARTIALLY VERIFIED / UNVERIFIED / CONTRADICTED status for each claim
- Evidence supporting the verification status

### Source Assessment
- Overall source quality rating (1-5)
- Concerns about any sources

### Answer Quality
- Whether the research answers the original question
- Missing elements

### Recommendations
- Suggested corrections or additions
- Areas needing more research

### Final Verdict
- Trustworthy and defensible? (Yes/With caveats/Needs work)

## Credibility Criteria Used

**High Credibility Sources:**
- Peer-reviewed research
- Official government/institutional data
- Established news organizations
- Primary sources and original documents

**Lower Credibility Sources:**
- Blogs and opinion pieces
- Social media
- Sites with heavy advertising
- Sources with clear conflicts of interest

## Integration with Main Agent

In production, the main agent would:
1. Receive research from `web-research-agent`
2. Send research to `credibility-agent` for verification
3. Use credibility feedback to refine or supplement research
4. Only include verified information in final reports